In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle

In [4]:
import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.layers import UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model
 
def _conv_block(inp, convs, skip=True):
	x = inp
	count = 0
	for conv in convs:
		if count == (len(convs) - 2) and skip:
			skip_connection = x
		count += 1
		if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x)
		x = Conv2D(conv['filter'],
				   conv['kernel'],
				   strides=conv['stride'],
				   padding='valid' if conv['stride'] > 1 else 'same',
				   name='conv_' + str(conv['layer_idx']),
				   use_bias=False if conv['bnorm'] else True)(x)
		if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
		if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
	return add([skip_connection, x]) if skip else x
 
def make_yolov3_model():
	image=Input(shape=(None, None, 3))
	x=_conv_block(image, [{'filter':32,'kernel':3, 'stride':1, 'bnorm': True, 'leaky': True, 'layer_idx': 0}, {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
								  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
								  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
	x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
						{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
						{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])
	x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
						{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])
	x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
						{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
						{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])
	for i in range(7):
		x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
							{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
	skip_36 = x
	x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])
	for i in range(7):
		x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
							{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
	skip_61 = x
	x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
	for i in range(3):
		x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
							{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])

	x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)

	yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
							  {'filter':  27, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)
	
	x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_61])

	x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)

	yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
							  {'filter': 27, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)

	x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_36])
	yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
							   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
							   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
							   {'filter': 27, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
	model = Model(image, [yolo_82, yolo_94, yolo_106])
	return model
 
class WeightReader:
	def __init__(self, weight_file):
		with open(weight_file, 'rb') as w_f:
			major,	= struct.unpack('i', w_f.read(4))
			minor,	= struct.unpack('i', w_f.read(4))
			revision, = struct.unpack('i', w_f.read(4))
			if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
				w_f.read(8)
			else:
				w_f.read(4)
			transpose = (major > 1000) or (minor > 1000)
			binary = w_f.read()
		self.offset = 0
		self.all_weights = np.frombuffer(binary, dtype='float32')
 
	def read_bytes(self, size):
		self.offset = self.offset + size
		return self.all_weights[self.offset-size:self.offset]
 
	def load_weights(self, model):
		for i in range(106):
			try:
				conv_layer = model.get_layer('conv_' + str(i))
				print("loading weights of convolution #" + str(i))
				if i not in [81, 93, 105]:
					norm_layer = model.get_layer('bnorm_' + str(i))
					size = np.prod(norm_layer.get_weights()[0].shape)
					beta  = self.read_bytes(size)
					gamma = self.read_bytes(size)
					mean  = self.read_bytes(size)
					var   = self.read_bytes(size)
					weights = norm_layer.set_weights([gamma, beta, mean, var])
				if len(conv_layer.get_weights()) > 1:
					bias   = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
					kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					kernel = kernel.transpose([2,3,1,0])
					conv_layer.set_weights([kernel, bias])
				else:
					kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					kernel = kernel.transpose([2,3,1,0])
					conv_layer.set_weights([kernel])
			except ValueError:
				print("no convolution #" + str(i))
 
	def reset(self):
		self.offset = 0

model = make_yolov3_model()
weight_reader = WeightReader('/content/drive/MyDrive/backup/yolo_custom_6000.weights')
weight_reader.load_weights(model)
model.save('/content/drive/MyDrive/Phase2/phase2_model.h5')

model = load_model('/content/drive/MyDrive/Phase2/phase2_model.h5',compile=True)

loading weights of convolution #0
loading weights of convolution #1
loading weights of convolution #2
loading weights of convolution #3
no convolution #4
loading weights of convolution #5
loading weights of convolution #6
loading weights of convolution #7
no convolution #8
loading weights of convolution #9
loading weights of convolution #10
no convolution #11
loading weights of convolution #12
loading weights of convolution #13
loading weights of convolution #14
no convolution #15
loading weights of convolution #16
loading weights of convolution #17
no convolution #18
loading weights of convolution #19
loading weights of convolution #20
no convolution #21
loading weights of convolution #22
loading weights of convolution #23
no convolution #24
loading weights of convolution #25
loading weights of convolution #26
no convolution #27
loading weights of convolution #28
loading weights of convolution #29
no convolution #30
loading weights of convolution #31
loading weights of convolution #32

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, None, None, 3 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, None, None, 3 0           bnorm_0[0][0]                    
______________________________________________________________________________________________

In [5]:
with open('modelsummary.txt', 'w') as f:
  model.summary(print_fn=lambda x: f.write(x + '\n'))

In [7]:
 
class BoundBox:
	def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
		self.xmin = xmin
		self.ymin = ymin
		self.xmax = xmax
		self.ymax = ymax
		self.objness = objness
		self.classes = classes
		self.label = -1
		self.score = -1
 
	def get_label(self):
		if self.label == -1:
			self.label = np.argmax(self.classes)
 
		return self.label
 
	def get_score(self):
		if self.score == -1:
			self.score = self.classes[self.get_label()]
 
		return self.score
 
def _sigmoid(x):
	return 1. / (1. + np.exp(-x))
 
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
	grid_h, grid_w = netout.shape[:2]
	#print(grid_h,grid_w)
	nb_box = 3
	netout = netout.reshape((grid_h, grid_w, nb_box, -1))
	#print(netout)
	nb_class = netout.shape[-1] - 5
	#print(nb_class)
	boxes = []
	netout[..., :2]  = _sigmoid(netout[..., :2])
	netout[..., 4:]  = _sigmoid(netout[..., 4:])
	netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
	netout[..., 5:] *= netout[..., 5:] > obj_thresh
 
	for i in range(grid_h*grid_w):
		row = i / grid_w
		col = i % grid_w
		for b in range(nb_box):
			objectness = netout[int(row)][int(col)][b][4]
			if(objectness.all() <= obj_thresh): continue

			x, y, w, h = netout[int(row)][int(col)][b][:4]
			#print(x,y,w,h)
			x = (col + x) / grid_w
			y = (row + y) / grid_h
			w = anchors[2 * b + 0] * np.exp(w) / net_w
			h = anchors[2 * b + 1] * np.exp(h) / net_h
			classes = netout[int(row)][col][b][5:]
			box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
			#box= BoundBox(x, y, w, h, objectness, classes)
			boxes.append(box)
   
	return boxes
 
def _interval_overlap(interval_a, interval_b):
	x1, x2 = interval_a
	x3, x4 = interval_b
	if x3 < x1:
		if x4 < x1:
			return 0
		else:
			return min(x2,x4) - x1
	else:
		if x2 < x3:
			 return 0
		else:
			return min(x2,x4) - x3
 
def bbox_iou(box1, box2):
	intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
	intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
	intersect = intersect_w * intersect_h
	w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
	w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
	union = w1*h1 + w2*h2 - intersect
	return float(intersect) / union
 
def do_nms(boxes, nms_thresh):
	if len(boxes) > 0:
		nb_class = len(boxes[0].classes)
	else:
		return
	for c in range(nb_class):
		sorted_indices = np.argsort([-box.classes[c] for box in boxes])
		for i in range(len(sorted_indices)):
			index_i = sorted_indices[i]
			if boxes[index_i].classes[c] == 0: continue
			for j in range(i+1, len(sorted_indices)):
				index_j = sorted_indices[j]
				if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
					boxes[index_j].classes[c] = 0
 
def load_image_pixels(filename, shape):
	image = load_img(filename)
	width, height = image.size
	image = load_img(filename, target_size=shape)
	image = img_to_array(image)
	image = image.astype('float32')
	image /= 255.0
	image = expand_dims(image, 0)
	return image, width, height
 
def get_boxes(boxes, labels, thresh):
	v_boxes, v_labels, v_scores = list(), list(), list()
	for box in boxes:
		for i in range(len(labels)):
			if box.classes[i] > thresh:
				v_boxes.append(box)
				v_labels.append(labels[i])
				v_scores.append(box.classes[i]*100)
	return v_boxes, v_labels, v_scores
 
def draw_boxes(filename, v_boxes, v_labels, v_scores):
	data = pyplot.imread(filename)
	pyplot.imshow(data)
	ax = pyplot.gca()
	for i in range(len(v_boxes)):
		box = v_boxes[i]

		y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
		width, height = x2-x1, y2-y1
		rect = Rectangle((x1, y1), width, height, fill=False, color='white')
		ax.add_patch(rect)
		label = "%s (%.3f)" % (v_labels[i], v_scores[i])
		pyplot.text(x1, y1, label, color='white')
	pyplot.show()
model = load_model('/content/drive/MyDrive/Phase2/phase2_model.h5')


In [8]:
def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
	new_w, new_h = net_w, net_h
	for i in range(len(boxes)):
		x_offset, x_scale = (net_w - new_w)/2./net_w, float(new_w)/net_w
		y_offset, y_scale = (net_h - new_h)/2./net_h, float(new_h)/net_h
		boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
		boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
		boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
		boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)
	
    

In [9]:
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
class_threshold = 0.01
labels = ["Eosinophil","Lymphocyte","Monocyte","Neutrophil"]

In [13]:
import numpy as np
import pandas as pd
import keras
from keras import applications
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Bidirectional, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization, Input, concatenate, Reshape, LSTM
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import xml.etree.ElementTree as ET
import sklearn
import itertools
import cv2
import scipy
import os
import csv
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm


In [19]:
actual=[]
predicted=[]
folder='/content/drive/MyDrive/Bloodcells/Test_phase2/TEST/'
for wbc_type in os.listdir(folder):

  if wbc_type in ['NEUTROPHIL']:
      label = 3
  elif wbc_type in ['EOSINOPHIL']:
      label = 0
  elif wbc_type in ['MONOCYTE']:
      label = 2
  elif wbc_type in ['LYMPHOCYTE']:
      label = 1
  else:
      label = 4
  x=folder+wbc_type
  for img in os.listdir(x):
    image1=x+'/'+img
    if(image1[len(image1)-3:]!="txt"):
      input_w, input_h = 608, 608
      image, image_w, image_h = load_image_pixels(image1, (input_w, input_h))
      yhat = model.predict(image)
      
      boxes = list()
      for i in range(len(yhat)):
        boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)

      correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
      do_nms(boxes, 0.5)
      v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
      
      if len(v_scores)>0:
        #print(v_labels[v_scores.index(max(v_scores))])
        i=v_scores.index(max(v_scores))
      #print(v_scores[i])
        predicted.append(labels.index(v_labels[i]))
      #print(v_boxes[i].xmin,v_boxes[i].xmax,v_boxes[i].ymin,v_boxes[i].ymax)
        actual.append(label)
      #val=val+1
      #print(image1)
      #draw_boxes(image1, v_boxes, v_labels, v_scores)
print(accuracy_score(actual,predicted))
print(classification_report(actual, predicted, target_names=labels))
print(confusion_matrix(actual,predicted))
      


0.8995884773662551
              precision    recall  f1-score   support

  Eosinophil       0.90      0.97      0.93       574
  Lymphocyte       1.00      1.00      1.00       620
    Monocyte       0.99      0.74      0.85       620
  Neutrophil       0.76      0.90      0.82       616

    accuracy                           0.90      2430
   macro avg       0.91      0.90      0.90      2430
weighted avg       0.91      0.90      0.90      2430

[[555   0   2  17]
 [  0 620   0   0]
 [  0   0 459 161]
 [ 63   0   1 552]]


In [ ]:
def correct_coords(coords):
  box=[]
  box.append(coords[0])
  c_x=coords[1]*320
  c_y=coords[2]*240
  w=coords[3]*320
  h=coords[4]*240

  box.append(c_x-w/2.)
  box.append(c_x+w/2.)
  box.append(c_y-h/2.)
  box.append(c_y+h/2.)
  return box


def get_coordinates(image1):
  line=open(image1,"r")
  Lines=line.readlines()
  for i in Lines:
    coord=i.split(' ')
    coord=[float(i) for i in coord]
    coord=correct_coords(coord)
  return coord

  




image1='/content/drive/MyDrive/Bloodcells/Test_phase2/Testing_data/EOSINOPHIL/_0_2107.txt'
box2=get_coordinates(image1)
print(box2)


[0.0, 190.07072, 264.39712000000003, 143.54616, 215.6028]


In [ ]:
def _interval_overlap(interval_a, interval_b):
	x1, x2 = interval_a
	x3, x4 = interval_b
	if x3 < x1:
		if x4 < x1:
			return 0
		else:
			return min(x2,x4) - x1
	else:
		if x2 < x3:
			 return 0
		else:
			return min(x2,x4) - x3
 
def box_iou(box1, box2):
	intersect_w = _interval_overlap([box1.xmin, box1.xmax], (box2[1], box2[2]))
	intersect_h = _interval_overlap([box1.ymin, box1.ymax], (box2[3], box2[4]))
	intersect = intersect_w * intersect_h
	w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
	w2, h2 = box2[2]-box2[1], box2[4]-box2[3]
	union = w1*h1 + w2*h2 - intersect
	return float(intersect) / union

In [ ]:
actual=[]
predicted=[]
val=0
default=[]
input_w, input_h = 608, 608
TN=0
TP=0
FP=0
FN=0
rows, cols = (5, 5)
arr = [[0]*cols]*rows
folder='/content/drive/MyDrive/Bloodcells/Test_phase2/Testing_data/'
for wbc_type in os.listdir(folder):

  if wbc_type in ['NEUTROPHIL']:
      label = 3
  elif wbc_type in ['EOSINOPHIL']:
      label = 0
  elif wbc_type in ['MONOCYTE']:
      label = 2
  elif wbc_type in ['LYMPHOCYTE']:
      label = 1
  else:
      label = 4
  x=folder+wbc_type
  for img in os.listdir(x):
    image1=x+'/'+img
    box2=[]
    box2=get_coordinates(image1)
    #print(box2)
    image1=image1[:34]+'TEST'+'/'+image1[59:len(image1)-3]+'jpeg'
    print(image1)

    image, image_w, image_h = load_image_pixels(image1, (input_w, input_h))
    yhat = model.predict(image)
    
    boxes = list()
    for i in range(len(yhat)):
      boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)

    correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
    do_nms(boxes, 0.5)
    v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
    #print(v_labels)
    #print(v_scores)
    
    if len(v_labels)>0:
      #print(v_labels[v_scores.index(max(v_scores))])
      #print(box2)
      i=v_scores.index(max(v_scores))
      iou=box_iou(v_boxes[i],box2)
      #print(iou)
      pred_1=labels.index(v_labels[i])
      if iou>.5 and v_scores[i]>.3 and int(box2[0])==pred_1:
        #print(box2[0],pred_1)
        
        TP=TP+1
      elif iou<.5 and int(box2[0])!=pred_1:
        FP=FP+1
        print(box2[0],pred_1)
      elif v_scores[i]<.3:
        FN=FN+1
      else:
        TN=TN+1
      


      #print(v_scores[i])
      #predicted.append(v_labels[i])
      #print(v_boxes[i].xmin,v_boxes[i].xmax,v_boxes[i].ymin,v_boxes[i].ymax)
      #actual.append(label)
      #val=val+1
      #print(image1)


/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_582.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_353.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_430.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_522.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_33.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_112.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_529.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_687.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_601.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_2599.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_3015.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_2295.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_1050.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_1022.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_3295.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_3271.jpeg
/c

Accuracy of Detection

In [ ]:
print((TP+TN)/(TP+TN+FP+FN))

0.9928057553956835


In [ ]:
actual=[]
predicted=[]
val=0
default=[]
input_w, input_h = 608, 608
TN=0
TP=0
FP=0
FN=0
rows, cols = (5, 5)
arr = [[0]*cols]*rows
folder='/content/drive/MyDrive/Bloodcells/Test_phase2/Testing_data/'
for wbc_type in os.listdir(folder):

  if wbc_type in ['NEUTROPHIL']:
      label = 3
  elif wbc_type in ['EOSINOPHIL']:
      label = 0
  elif wbc_type in ['MONOCYTE']:
      label = 2
  elif wbc_type in ['LYMPHOCYTE']:
      label = 1
  else:
      label = 4
  x=folder+wbc_type
  for img in os.listdir(x):
    image1=x+'/'+img
    box2=[]
    box2=get_coordinates(image1)
    #print(box2)
    image1=image1[:34]+'TEST'+'/'+image1[59:len(image1)-3]+'jpeg'
    print(image1)

    image, image_w, image_h = load_image_pixels(image1, (input_w, input_h))
    yhat = model.predict(image)
    
    boxes = list()
    for i in range(len(yhat)):
      boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)

    correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
    do_nms(boxes, 0.5)
    v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
    #print(v_labels)
    #print(v_scores)
    
    if len(v_labels)>0:
      #print(v_labels[v_scores.index(max(v_scores))])
      #print(box2)
      i=v_scores.index(max(v_scores))
      iou=box_iou(v_boxes[i],box2)
      #print(iou)
      pred_1=labels.index(v_labels[i])
      if iou>.5 and v_scores[i]>.4 and int(box2[0])==pred_1:
        #print(box2[0],pred_1)
        a[int(box2[0])][0]=a[int(box2[0])][0]+1
        TP=TP+1
      elif iou<.5 and int(box2[0])!=pred_1:
        FP=FP+1
        a[int(pred_1)][1]=a[int(pred_1)][1]+1
        print(box2[0],pred_1)
      elif v_scores[i]<.4:
        FN=FN+1
        a[int(pred_1)][2]=a[int(pred_1)][2]+1
      else:
        TN=TN+1
        a[int(pred_1)][3]=a[int(pred_1)][3]+1
      


      #print(v_scores[i])
      #predicted.append(v_labels[i])
      #print(v_boxes[i].xmin,v_boxes[i].xmax,v_boxes[i].ymin,v_boxes[i].ymax)
      #actual.append(label)
      #val=val+1
      #print(image1)


/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_582.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_353.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_430.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_522.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_33.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_112.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_529.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_687.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_601.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_2599.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_3015.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_2295.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_1050.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_1022.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_3295.jpeg
/content/drive/MyDrive/Bloodcells/TEST/LYMPHOCYTE/_0_3271.jpeg
/c

White Blood Cell Subtypes Individual Accuracy

In [ ]:
for i in range(4):
  print(labels[i])
  print("Accuracy:",(a[i][0]+a[i][3])/(a[i][0]+a[i][3]+a[i][1]+a[i][2]))
  print("Precision:",(a[i][0])/(a[i][0]+a[i][1]))
  print("Recall:",(a[i][0])/(a[i][0]+a[i][2]))






Eosinophil
Accuracy: 0.974025974025974
Precision: 0.9701492537313433
Recall: 1.0
Lymphocyte
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
Monocyte
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
Neutrophil
Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [ ]:
tp=0
fp=0
fn=0
tn=0
for i in range(4):
  tp=a[i][0]+tp
  fp=a[i][1]+fp
  fn=a[i][2]+fn
  tn=tn+a[i][3]

print("Accuracy:",(tp+tn)/(tp+tn+fp+fn))
  


Accuracy: 0.9928057553956835
